# Imports

In [1]:
! pip install transformers
! pip install torch-scatter -f https://data.pyg.org/whl/torch-1.9.0+cpu.html

     |████████████████████████████████| 2.9 MB 14.6 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 895 kB 26.3 MB/s 
     |████████████████████████████████| 3.3 MB 33.9 MB/s 
     |████████████████████████████████| 636 kB 63.5 MB/s 
     |████████████████████████████████| 109 kB 51.8 MB/s 
     |████████████████████████████████| 546 kB 68.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in links: https://data.pyg.org/whl/torch-1.9.0+cpu.html
     |████████████████████████████████| 300 kB 13.0 MB/s 


In [2]:
from transformers import pipeline
import pandas as pd

# Load model

In [3]:
model = pipeline("table-question-answering", 
                 model='google/tapas-large-finetuned-wtq')

Downloading:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/490 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/154 [00:00<?, ?B/s]

# Inference

In [4]:
data = pd.read_csv('http://data.insideairbnb.com/singapore/sg/singapore/2021-07-19/visualisations/listings.csv').sample(10).reset_index(drop=True)
query = ["Which listing is the most expensive?", "Which region has the most listings?", "What is the average number of reviews per month?"]

# data = {'Actors': ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"], 'Number of movies': ["87", "53", "69"]}
# query = ["What is the name of the first actor?", "How many movies has George Clooney played in?", "What is the total number of movies?"]

table = pd.DataFrame.from_dict(data).applymap(str)

table

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,20181917,Clean Dorms - Steps from Chinatown MRT! (Per Bed),100353323,Royal,Central Region,Outram,1.28484,103.84285,Hotel room,31,1,1,2019-09-23,0.05,8,365,0,nan
1,46423926,Hotel Family Room in City Center,368878101,The Quay,Central Region,Kallang,1.3114,103.86233,Private room,119,1,3,2021-02-16,0.45,4,365,3,nan
2,14719882,"COMMON ROOM AVAILABLE @ CLEMENTI,",89770625,Sangi,West Region,Clementi,1.30367,103.7695,Private room,55,2,7,2018-03-14,0.12,2,293,0,nan
3,24682695,Premium Room with Balcony in Orchard (300 sqft),111065775,Joyce,Central Region,River Valley,1.2990000000000002,103.83634,Hotel room,139,6,0,nan,nan,17,179,0,nan
4,34237286,Family Room for 14 (No Breakfast),258477842,Seven,Central Region,Rochor,1.3054299999999999,103.85329,Private room,359,1,4,2019-09-21,0.15,11,291,0,nan
5,18283249,Family Tent Camping by the beach,126416861,Gary,East Region,Bedok,1.30785,103.93181,Entire home/apt,230,1,0,nan,nan,1,0,0,nan
6,17740089,Orchard Road 2 single beds,119740557,Nicholas,Central Region,Newton,1.30571,103.84076999999999,Private room,65,3,9,2017-07-21,0.17,3,89,0,nan
7,17004808,Cosy Room in Upper Thomson Road,33151277,Tarcia,North-East Region,Ang Mo Kio,1.3746399999999999,103.82849,Private room,35,90,1,2017-06-14,0.02,1,358,0,nan
8,50982468,"The Good Room, Nearby DT Rochor 02",374499296,Yiyi,Central Region,Rochor,1.3052,103.85204,Private room,116,1,0,nan,nan,31,352,0,nan
9,30596829,"LUSH 2 BR DERBYSHIRE APARTMENT, NOVENA.",156409670,Tia,Central Region,Novena,1.3208,103.84574,Entire home/apt,220,80,0,nan,nan,144,364,0,nan


In [5]:
output = model(table=table, 
               query=query,
               padding='max_length',
               truncation=True)

result = pd.DataFrame(output)

result['Query'] = query
result['Answer'] = result['answer'].copy()
result = result[['Query', 'Answer']]

result

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


,Query,Answer
0,Which listing is the most expensive?,Family Room for 14 (No Breakfast)
1,Which region has the most listings?,Central Region
2,What is the average number of reviews per month?,"AVERAGE > 0.05, 0.45, 0.12, 0.15, 0.17, 0.02"
